# Log Data Example

<div class="alert alert-block alert-info">
    
This example demonstrates how to import and process data from the *[CARWatch App](https://play.google.com/store/apps/details?id=de.fau.cs.mad.carwatch)* used to record logs during cortisol awakening response (CAR) assessment studies at home.
  
</div>

## Imports and Helper Functions

In [ ]:
from pathlib import Path

import pandas as pd
import numpy as np

import biopsykit as bp
from biopsykit.carwatch_logs import LogData

import matplotlib.pyplot as plt
import seaborn as sns

import ipywidgets as widgets

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
palette = bp.colors.fau_palette
sns.set_theme(
    context="notebook", 
    style="ticks", 
    font="sans-serif",
    palette=palette
)

plt.rcParams['figure.figsize'] = (8, 4)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

palette

In [ ]:
log_path = bp.example_data.get_car_watch_log_path_example()
# Alternatively: Use your own data
# log_path = Path("<path-to-log-files-folder>")

## Loading/Saving

Log files can either be imported by (1) loading the `.zip` files obtained by the *CARWatch App* or (2) by loading a subject folder with `.csv` files.

For (1), the file structure is assumed to look like:
```
. (base folder)
├── logs_ABC12.zip
├── logs_DEF34.zip
├── logs_GHI56.zip
├── logs_JKL78.zip
...
```

For (2), the file structure is assumed to look like:
```
. (base folder)
├── ABC12/
│   ├── carwatch_abc12_20200101.csv
│   ├── carwatch_abc12_20200102.csv
│   └── carwatch_abc12_20200103.csv
├── DEF34/
│   ├── carwatch_def34_20200201.csv
│   ├── carwatch_def34_20200202.csv
│   └── carwatch_def34_20200203.csv
...
```

### Load Single Subject

Example Log Data:

In [ ]:
# Load log data from folder:
folder_path = bp.example_data.get_car_watch_log_path_example()
# Alternatively: Use your own data
# folder_path = Path("<path-to-log-file-folder>")
logs = bp.io.carwatch_logs.load_log_one_subject(folder_path)

# Load log data from zip file:
zip_path = bp.example_data.get_car_watch_log_data_zip_path_example()
# Alternatively: Use your own data
# zip_path = Path("<path-to-log-zip-file>")
logs = bp.io.carwatch_logs.load_log_one_subject(zip_path)

logs

### Save Log Data as csv

In [ ]:
# uncomment to save data
# bp.io.carwatch_logs.save_log_data(logs, log_path)

### Load All Subjects

Load Log files stored in subfolders and concatenate them into one large dataframe:

In [ ]:
log_path = bp.example_data.get_car_watch_log_path_all_subjects_example()
# Alternatively: Use your own data
# log_path = Path("folder-path-to-log-data")

In [ ]:
df_logfiles = bp.io.carwatch_logs.load_logs_all_subjects(log_path, has_subject_folders=True)
# Alternatively: If Log files are provided as zip files (or were already exported as combined csv files) instead of being stored in subfolders, you can load them like this:
# df_logfiles = bp.log_data.io.load_logs_all_subjects(log_path, has_subfolder=False)

df_logfiles

### Save All Subjects

In [ ]:
# Uncomment to save log data
# bp.io.carwatch_logs.save_log_data(df_logfiles, log_path.joinpath("export_logs.csv"))

### Create `LogData` object

The `LogData` object can help to better handle log data

In [ ]:
# select one subject using dataframe cross-select
df = df_logfiles.xs("DE34F")

logdata = LogData(df)

In [ ]:
logdata

## Filtering/Selecting

### Filter by Action

In [ ]:
bp.carwatch_logs.log_data.get_logs_for_action(logdata, "barcode_scanned")

### Extract Extras from Log Actions

In [ ]:
# returns extras for the first log entry with specific action
bp.carwatch_logs.log_data.get_logs_for_action(logdata, "barcode_scanned")

### Filter by Date

In [ ]:
bp.carwatch_logs.log_data.get_logs_for_date(logdata, "2019-12-06")

### Split Log Data into different Nights

In [ ]:
list_nights = bp.carwatch_logs.log_data.split_nights(logdata)

In [ ]:
list_nights[1]

## Log Statistics

In [ ]:
log_stats = bp.carwatch_logs.log_statistics.LogStatistics(log_path)

### Android Versions

In [ ]:
log_stats.android_versions()

In [ ]:
fig, ax = plt.subplots()
log_stats.get_plot('android_version', ax=ax)

### Smartphone Manufacturer

In [ ]:
log_stats.manufacturer()

In [ ]:
fig, ax = plt.subplots()
log_stats.get_plot('manufacturers', ax=ax)

### Recording Days

In [ ]:
log_stats.days()

In [ ]:
fig, ax = plt.subplots()
log_stats.get_plot('days', ax=ax)

## Widgets

In [ ]:
out_logfile = widgets.Output()

@out_logfile.capture(clear_output=True)
def callback_log_file_subject(change):
    subject_id = change['new']
    display(df_logfiles.xs(subject_id))

In [ ]:
from biopsykit.carwatch_logs.widgets import log_file_subject_dropdown
log_file_subject_dropdown(log_path, input_type='folder', value_type="subject_id", callback=callback_log_file_subject)

In [ ]:
out_logfile

In [ ]:
out_action = widgets.Output()

@out_action.capture(clear_output=True)
def callback_log_action(change):
    action = change['new']
    display(df_logfiles[df_logfiles["action"] == action])

In [ ]:
from biopsykit.carwatch_logs.widgets import action_dropdown_widget
action_dropdown_widget(df_logfiles.xs('DE34F'), callback=callback_log_action)

In [ ]:
out_action

In [ ]:
df_logfiles.loc[pd.IndexSlice[:, df_logfiles.index.get_level_values("time") == "2019-12-07"], :]

In [ ]:
out_day = widgets.Output()

@out_day.capture(clear_output=True)
def callback_log_day(change):
    day = change['new']
    print(day)

In [ ]:
from biopsykit.carwatch_logs.widgets import day_dropdown_widget
day_dropdown_widget(df_logfiles.xs('DE34F'), callback=callback_log_day)

In [ ]:
out_day